In [1]:
import json
import random
from uuid import uuid4
import pandas as pd
from datasets import load_dataset
from collections import defaultdict

ds = load_dataset("rajpurkar/squad")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [3]:
ds["train"][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [4]:
# ds["train"][0]
# {'id': '5733be284776f41900661182',
#  'title': 'University_of_Notre_Dame',
#  'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
#  'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
#  'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}
data = defaultdict(list)
for i in range(len(ds["train"])):
    data[ds["train"][i]["context"]].append(ds["train"][i]["question"])
for i in range(len(ds["validation"])):
    data[ds["train"][i]["context"]].append(ds["train"][i]["question"])
keys = list(data.keys())

In [5]:
count = 0
for key in keys:
    if len(set(data[key])) >= 4:
        count += 1
print(count)


15268


In [6]:
for key in keys:
    if len(set(data[key])) < 4:
        del data[key]
    

In [7]:
sampled_data = defaultdict(list)
for key in data.keys():
    sampled_data[key] = random.sample(set(data[key]), 4)

/tmp/ipykernel_62768/3753069355.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  sampled_data[key] = random.sample(set(data[key]), 4)


In [8]:
with open("data.jsonl", "w") as f:
    for key in sampled_data.keys():
        f.write(json.dumps({"context": key, "questions": sampled_data[key]}) + "\n")

In [10]:
# {
#    "query_id": "<query id>",
#    "query": "<query text>",
#    "positive_passages": [
#      {"docid": "<passage id>", "title": "<passage title>", "text": "<passage body>"},
#      ...
#    ],
#    "negative_passages": [
#      {"docid": "<passage id>", "title": "<passage title>", "text": "<passage body>"},
#      ...
#    ]
# }
with open("dataset.jsonl", "w") as f:
    for key in sampled_data.keys():
        for question in sampled_data[key]:
            f.write(json.dumps({"query_id": str(uuid4()), "query": question, "positive_passages": [{"docid": str(uuid4()), "title": "title", "text": key}], "negative_passages": []}) + "\n")
        # break